# Introduction
Digital engineering (DE) is a technology that can
- increase the visibility of what is most urgent and important,
- identify how changes that are introduced affect a whole system, as well as parts of it, and
- enable stakeholders of a system to retrieve timely information about the status of a product moving through the development lifecycle at any point in time.

**Digital Perspective:** DE was initially conceived as the methodology and implementation for designing and establishing digital products or systems.  

<center>
    <img src="img/19.png" alt="Alt text" width="500">
</center>


**System engineering (SE) perspective:** How modern digital approaches facilitate the development of, from text-based systems engineering (document-centric) to model-based systems engineering (model-centric).

<center>
    <img src="img/20.jpg" alt="Alt text" width="600">
</center>

**Continues to evolve:** The varying colors of these keywords indicate shifts in thematic focus over time. Earlier approaches primarily concentrated on the use of digital technologies to address issues during the design phase, and keywords such as “CAD”, “CAE”, “FEM”, “design” were more frequently used in this phase. With the development of information technology, the dominant trends have naturally shifted towards more advanced digital technologies and systematic approaches. Recently, keywords like “digital twin”, “artificial intelligence”, “systems engineering”, and “model-based systems engineering” have become prevalent topics of discussion.

<center>
    <img src="img/21.jpg" alt="Alt text" width="900">
</center>

**Expected Benefits:** DE holds great expectations and benefits: (1) costs minimization; (2) global optimizations facilitation; and (3) prompt rapid responses to requirements.

<center>
    <img src="img/22.jpg" alt="Alt text" width="400">
</center>



## Fundamentals of Digital Engineering:
Digital engineering is an integrated digital approach that uses authoritative sources of systems data and models as a continuum across disciplines to support lifecycle activities from concept through disposal. At its core, digital engineering evolves creating comprehensive digital (computer) models that represent every characteristic of a complex product or system that is to be developed. Originally designed for hardware-intensive lifecycle management, digital engineering also proves highly effective for software applications. Complex systems often depend on multiple subsystems operating simultaneously and in concert, known as a system of systems (SoS). This concept applies to both software and hardware-intensive systems. While integration challenges exist for both, the process for managing and scheduling hardware and software integration can differ significantly. Digital engineering addresses these challenges by providing a cohesive framework that ensures efficient and coordinated lifecycle management across all system components.

<center>
    <img src="img/01.png" alt="Alt text" width="900">
</center>

Digital engineering is sometimes referred to as “digital thread”, which encompasses a broader scope. Although the novelty of both terms has led to some debate, they are fundamentally extensions of product lifecycle management. This practice involves creating and storing a system's lifecycle artifacts in digital form, which can be updated as the system evolves. Both digital thread and digital engineering involve a single source of truth, referred to as the authoritative source of truth (ASoT). This ASoT contains all lifecycle artifacts in a centralized repository, ensuring that stakeholders work from the same models rather than multiple copies. The emerging processes of digital thread and digital engineering aim to address the difficulties of managing complex and evolving technologies over their lifecycles.

Digital engineering has an ambitious scope: managing the entire lifecycle of a system of systems, including both hardware systems and software components. Its emphasis on simulation and modeling offers significant benefits in hardware engineering alongside software engineering. By promoting the use of virtual models in tandem with, or prior to, constructing physical systems, digital engineering aims to improve efficiency, efficacy, and success. An offshoot of Model-Based Systems Engineering (MBSE), digital engineering seeks to accelerate the development of complex products, systems, and sub-systems. It achieves this by integrating various engineering disciplines and addressing each unique engineering challenge comprehensively. 

One application example of digital engineering is the digital twin, which is a software simulation that mirrors the operation of a physical system. These digital twins can also simulate software, reflecting the interdependence of physical and software components. Digital twins can be used to reduce the time, money, and power required to test and iterate on the design of novel technologies. In its current state, digital engineering is new and relatively unrecognized as an industry term.  The introduction of new standards and practices from digital engineering may come full-circle and influence the way that software is currently built, placing a larger emphasis on the modeling and simulation of software systems.

The objective of this training course is to familiarize the audience with the concepts and tools necessary for digital engineering, as well as the current challenges in the field. In contrast to traditional text/image-based course material, this course utilizes Jupyter notebooks to demonstrate the concepts and tools interactively. The format allows participants to practice these concepts at their own pace on an open-source platform. The course modules are designed to cater to a diverse audience (novices, learners and practitioners) accommodating various levels of experience and interest.

**Quick tour:** JupyterLab and Tools

<center>
    <img src="img/11.jpg" alt="Alt text" width="700">
</center>


### <font color = 'red'>Quick Tutorial

<center>
    <img src="img/26.png" alt="Alt text" width="500">
</center>

#### This example demonstrates how to run interactive modules on our JupyterHub platform. Press ▶️ to begin the interactive session and insert your name.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from ipywidgets import interact, FloatSlider, IntSlider
import ipywidgets as widgets
from IPython.display import display, FileLink
import os
from ipywidgets import fixed

print('Code Running...')
user_name = 'Unknown'

# Create a text input widget
name_input = widgets.Text(
    value='',
    placeholder='Type your name here',
    description='Insert Name',
    disabled=False
)

# Create an output widget
output = widgets.Output()

# Function to update the global variable and clear output
def update_name(change):
    global user_name
    user_name = change['new']
    with output:
        output.clear_output()  # Clear the previous output
        print(f"Name '{user_name}' will be provided to the following code cell.")

# Linking the text widget to the update function
name_input.observe(update_name, names='value')

# Display the text input widget and output widget
display(name_input, output)

#### This code cell below serves to showcase interactive tools for learning modules, rather than teaching specific concepts.
#### Press ▶️ to start interactive plot.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from ipywidgets import interact, IntSlider, FloatSlider
print('You can adjust the Line Degree of Complexity, and the Noise Level in Data Generated\n\nDrag Slider and Interact !')
# Initialize x data
x = np.linspace(-3, 3, 50)
np.random.seed(0)  # Set a fixed seed for reproducibility

# Function to generate data
def generate_data(noise, x=x):
    return x**2 + np.random.randn(50) * noise

# Cache the initial data
initial_noise = 0.8  # Initial noise level
y = generate_data(initial_noise)  # Initial data generation

# Function to plot polynomial regression
def plot_poly_regression(degree, y):
    # Reshape x for sklearn compatibility
    X = x[:, np.newaxis]
    
    # Create polynomial features
    poly = PolynomialFeatures(degree=degree)
    X_poly = poly.fit_transform(X)
    
    # Fit a linear model
    model = LinearRegression()
    model.fit(X_poly, y)
    
    # Predict and plot
    y_pred = model.predict(X_poly)
    plt.figure(figsize=(5, 5))
    
    
    # Adding the user's name as a transparent text in the background
    plt.text(0.5, 0.92, user_name + '\'s Plot', fontsize=22, color='#2F65A7', ha='center', va='center',
             alpha=1, transform=plt.gca().transAxes)  # Using axis transform for relative positioning
    plt.scatter(x, y, color='#006EB2', label='Data points')
    plt.plot(x, y_pred, color='#9A3324', label=f'Polynomial Degree {degree}', linewidth=2)
    plt.ylim([-40 - 10, 40 + 10])
    # plt.title(f"Polynomial Degree {degree} Fit with Noise Level {initial_noise}")
    # plt.legend()
    plt.grid(True)
    plt.show()

# Widgets for degree and noise adjustment
degree_slider = IntSlider(value=1, min=1, max=15, step=1, description="Degree")
noise_slider = FloatSlider(value=initial_noise, min=0.1, max=10, step=0.1, description="Noise Level")

# Track the previous noise level to decide whether to regenerate data
previous_noise = initial_noise

def interactive_update(degree, noise):
    global y, previous_noise
    if noise != previous_noise:
        y = generate_data(noise)  # Regenerate data if noise level has changed
        previous_noise = noise
    plot_poly_regression(degree, y)

# Interactive widget setup
interact(interactive_update,
         degree=degree_slider,
         noise=noise_slider);


### Module 1: Data
To begin a successful digital engineering journey, it is important to recognize that data is the linchpin driving this transformation. The true power of digital engineering lies not in adopting the latest digital technologies but in making data-driven decisions. Data can be collected from anywhere and everywhere, but the utility of digital engineering is as good as data quantity and quality. It serves as the compass that guides the transformation, enabling informed decision-making, product innovation, service enhancement, and efficient process automation.

<center>
    <img src="img/23.jpg" alt="Alt text" width="300">
</center>


#### Concepts:
- **Data sources:** Data sources refer to the origins from which data is obtained, including databases,
sensors, and external services, providing raw information for analysis and processing. They can
be structured, semi-structured, or unstructured, depending on the format and organization of the
data.

- **IoT:** IoT refers to a network of interconnected devices embedded with
sensors, software, and other technologies, enabling them to collect, exchange, and act on data
over the internet. These devices range from household items to industrial machines, enhancing
automation and data-driven decision-making.

- **Data cleansing:** Data cleansing can involve filtering out noise and removing outliers from
collected data to ensure its their quality and reliability. This process ensures that the data used for
analysis is accurate, complete, and relevant.
    
- **Data storage, security, and accessibility:** Data storage involves saving data in physical or
cloud-based systems, while data security focuses on protecting this data from unauthorized
access and breaches. Data accessibility ensures that authorized users can efficiently retrieve and
use the stored data whenever needed.
    
- **Data Usage:** Data usage encompasses leveraging data to build models, perform simulations, and
apply machine learning techniques to optimize designs and processes. This includes analyzing
sensor data, predicting system behaviors, and improving operational efficiency through data-
driven decision-making and automation.
    
<center>
    <img src="img/conceptsFigure.svg" alt="Alt text" width="900">
</center>



#### User Stories:
#### Aerotech Data Collection Platform:
- [Polymer 3D Printer](Modules/3.%20Advanced%20Topics/3.1.%20Aerotech%20Data%20Collection%20Platform/Aerotech%20Data%20Collection.ipynb) - This module preprocesses sensor data from a custom-built Fused Filament Fabrication (FFF) 3D printer for modeling the extrusion process, incorporating feedback on motion, temperature, pressure, force, and cooling dynamics. 

#### Summary Data:
- Millions of data points are created at every stage of the product’s lifecycle.
- Making sense of that data and using it when needed and for the right purpose is what matters.
- Strategizing about how, where, and what kind of data to collect is very important.

#### Assessment:
- [Take a quiz](http://example.com/quiz)


### Module 2: Model Based Definition

Model-based definition (MBD) is a process that uses 3D models within CAD software to define product assemblies and individual components comprehensively. The model is annotated with non-geometric information, such as dimensions, materials, finishes, tolerances, and symbols, that are important for manufacturing and inspection. The model becomes the source of authority for all engineering activities and is used in downstream processes like manufacturing, inspection, and assembly. MBD can offer several benefits, including: (a) Reduced engineering documentation time: Compared to drawing-centric workflows, MBD can reduce the time spent on engineering documentation, (b) More efficient drafting process: When designers designate critical dimensions, weld callouts, and notes in the model before passing it on to drafters, there's less back-and-forth between the two, allowing drafters to have all the information they need as soon as it's ready, (c) Improved communication and collaboration: MBD can improve communication and collaboration among stakeholders, and (d) Cost savings: efficiencies gained through MBD lead to substantial cost savings.

<center>
<img src="img/05.jpg" alt="Alt text" width="500"></center>

#### Concepts:
- **Conceptual Model:** A conceptual model is an abstract representation of a system or process, designed to help understand, predict, or communicate its essential features. It often involves diagrams, flowcharts, or other visual tools to outline the relationships and interactions within the system, without diving into detailed specifics.

- **Fully Defined Model:** A fully defined model includes all the specific details necessary for implementation or analysis. In engineering and design, this means all dimensions, materials, tolerances, and other parameters are specified, allowing for accurate manufacturing, simulation, or evaluation.

- **CAD/Scanning Tools:** CAD (Computer-Aided Design) tools are software applications used to create, modify, analyze, or optimize a design. These tools enable the creation of detailed 2D or 3D models of physical objects, which can be used for engineering, manufacturing, and simulation purposes. 
    
    Scanning tools, often used in conjunction with CAD, capture the physical dimensions and features of real-world objects to create precise digital representations.

    <img src="img/17.png" alt="Alt text" width="300">

- **CAM Tools:** CAM (Computer-Aided Manufacturing) tools use CAD data to control and automate manufacturing processes. These tools generate the instructions (such as G-code) that guide machinery in fabricating parts, ensuring precision and efficiency in production. CAM tools integrate design and manufacturing processes, facilitating the transition from digital models to physical products.

    <img src="img/18.png" alt="Alt text" width="300">
    
<center>
<img src="img/Model Based Definition.svg" alt="Alt text" width="500"></center>

#### User Stories:
#### The "M" Block

<center>
<img src="img/Model Based Engineering Example.png" alt="Model Based Engineering Example" width="800 uw">
</center>

Standard CAD formats can store data such as dimentions, tolerance, material, and manufacturing method. Projects using a model based definition need to store more information than what standard CAD formats allow. This 'M' block model was designed for a model based definition. It contains the standard CAD information such as dimentions, tolerance, material, and manufacturing method; but it also contains information about the manufacturing process such as the maximum recored deviation or final dimentions. It also contains sensor information from the manufacturing process, such as motor current feedback, and 3D scan data. It can also store data for non engineering needs such as final cost, packaging requirements, and final customer.

To achieve this we created our own file format (.mbe) that stores all this data. The benefit of this format is you can load a specific part of the file when needed. If the engineers need to view the 3d scan, they open the .mbe file and the program will only load the 3D scan it wont load the other information. If sales people need to view the final cost they can select the .mbe file and their program will only load the infromation necessary for sales. The .mbe file is the authoritative source of the product defiition.

#### Assessment:
- [Take a quiz](http://example.com/quiz)

### Module 3: Model Based Enterprise/Engineering (MBE)
Historically, the industry relied on drawings to communicate requirements for manufacturing components and systems. The decentralization of manufacturing systems has amplified the challenge of collecting and communicating the product and process specifications needed to make decisions about design, production, and supply chains tasks while delivering products to market. Moving away from a reliance on drawings, MBE leverage computer-based technology to design, price, and manufacture items in digital-centric environment. MBE is a fully integrated and collaborative environment built on detailed 3D product definition shared across the enterprise; to enable rapid, seamless, and affordable deployment of products from concept to disposal.  The foundational elements of MBE are: A single digital master data set containing the 3D model and all needed product data in a managed secure and controlled environment that supports maximum data reuse for all aspects of acquisition, maintenance and operations.

<center>
    <img src="img/04.jpg" alt="Alt text" width="400">
</center>

#### Concepts:
- **Authoritative source of truth (ASoT):** An authoritative source of truth is a system or repository that is considered the most reliable source for information, such as identity information about an individual. It can also be a combination of multiple data sources. ASoTs are also known as primary-source data.

<img src="img/06.jpg" alt="Alt text" width="450">

- **Configuration management (CM):** To be effective, the authoritative source of truth must be established by the organization using clear standards, procedures and guidelines that promote its inherent value as well as institutionally protect its integrity. Governance assists in ensuring the models are properly managed, and that stakeholders are apprised of the correct ways to collect, share, and maintain the models and data accurately.

<img src="img/07.1.jpg" alt="Alt text" width="500">

- **Product Lifecycle Management (PLM):** The concept of Product Lifecycle Management (PLM) is based on the observation that all products have a similar lifecycle and undergo similar stages. These stages include inception, design, manufacture, assembly, service, and disposal. PLM is not a definition of a piece, assembly or product, but an overview of management aspects for handling the product from the first idea through development, production, service, and disposal.

<img src="img/08.jpg" alt="Alt text" width="600">

#### User Stories:
#### Cost Estimation Analysis

- [3D Printing Cost Estimate](Modules/2.%20Learning%20Modules/2.3.%20Cost%20Estimation%20Analysis/Cost%20Estimation%20using%20GCode.ipynb) - This modules of cost analysis calculates the cost of a 3D printed part which involves considering labor, printer operation costs, material choice, and the part’s volume and shape, all critical for determining total production costs and ensuring economic viability.

#### Summary MBE:
- **Data:** Refers to the source data itself.
- **Management:** Refers to the need to organize the massive amounts of data. This can be applied to the overall data set or down to the organizational schema used in the annotated models making up the Model Based Definition (MBD).
- **Secure & Controlled:** Refers to the ability to control access and secure the data once it has been managed.

#### Assessment:
- [Take a quiz](http://example.com/quiz)

### Module 4: Modeling and Simulation
Modeling and simulation involve the creation of models (e.g., physical, mathematical, behavioral, or logical representation of a system, entity, phenomenon, or process) as the foundation for simulations to develop data utilized for managerial or technical decision making.
<center>
<img src="img/24.jpg" alt="Alt text" width="600">
</center>


<center>
<img src="img/25.jpg" alt="Alt text" width="400">
</center>

#### Concepts:
- **Traditional Engineering Analysis (FEA):** Finite Element Analysis (FEA) is a computational method used to predict how products will react to forces, vibration, heat, and other physical effects, allowing for optimized design and validation before physical prototypes are made.

<center>
<img src="img/13.png" alt="Alt text" width="500">
</center>

- **Advanced multi-physics modeling and simulation:** This involves the simultaneous simulation of multiple interacting physical phenomena (e.g., thermal, mechanical, electrical) to predict the behavior of complex systems more accurately.

<center>
<img src="img/14.png" alt="Alt text" width="500">
</center>

- **CAM tools and build simulations:** Computer-Aided Manufacturing (CAM) tools assist in the planning, control, and management of manufacturing operations, while build simulations predict the manufacturing process and outcomes, ensuring efficiency and precision.

- **Data driven models:** These models use data analytics and machine learning algorithms to predict system behavior and performance based on historical and real-time data, enhancing decision-making and optimization.

<center>
<img src="img/15.png" alt="Alt text" width="500">
</center>

- **Physics based data models:** These models combine physical laws and empirical data to simulate and predict the behavior of systems under various conditions, providing a more comprehensive understanding of system performance.
    
- **Generative models:** Generative models automatically create optimized designs based on specified constraints and performance criteria, enabling innovative solutions that might not be discovered through traditional methods.

- **System Models:** System models represent the interactions and relationships between different components of a complex system, allowing for comprehensive analysis and optimization of the entire system's performance.

<center>
<img src="img/16.png" alt="Alt text" width="500">
</center>

#### User Stories:
####  Graph Neural Netowrks in Accelerating Finite Element Analysis:
- [Accelerated Finite Element with GNN](Modules/3.%20Advanced%20Topics/3.2.%20Material%20Properties%20Finite%20Element%20Analysis/Material%20Properties%20FEA.ipynb) - This module demonstrates how our Graph Neural Network accelerates finite element analysis by efficiently predicting material behavior under mechanical loads.

#### Assessment:
- [Take a quiz](http://example.com/quiz)



### Module 5: Model Based Process Control

Model-based process control is a mathematical and visual technique to addressing the challenges associated with designing complex control. This technique involves two key aspects. (a) The control analysis, representing the modeling and simulation aspects of the system including tuning and configuration, and automatic code generation. Each block of the model contains mathematical equations that emulate the behavior of the physical system. (b) The system and software, representing continuous validation and verification through testing, and deployment of the system. In this regard, when model-based control is used efficiently, it offers a single design platform to enhance overall system design. Through virtual prototyping, system engineers can easily see whether the whole system is working as intended, even before the hardware is manufactured and available for testing.


#### Concepts:
<center>
    <img src="img/03.jpg" alt="Alt text" width="400">
</center>


####  Model Predictive Control
- [Model Predictive Control](Modules/2.%20Learning%20Modules/2.6.%20Model%20Predictive%20Control/Model%20Predictive%20Control.ipynb) - This module demonstrates calculating a single time step's move trajectory using the same system as the Dahlin controller.

### Module 6: Digital Twins
A digital twin is a virtual replica of a real-world object, system, or process that serves multiple purposes, including simulation, testing, monitoring, and maintenance. Digital twins can encompass the entire lifecycle of an object and use real-time data from sensors to simulate behavior and monitor operations. By incorporating machine learning and advanced analytics, digital twins provide valuable insights and support informed decision-making, enhancing efficiency, and optimizing performance throughout the lifecycle.

<center>
    <img src="img/12.jpg" alt="Alt text" width="250">
</center>

#### Concepts:
- **Product Twin:** A Product Twin is a digital replica of a physical product used to analyze and optimize the product's lifecycle from design through decommissioning. It enables manufacturers to monitor performance and predict maintenance needs.

- **System Twin:** A System Twin represents a digital model of an interconnected system of products and processes, providing insights into the overall system performance and interactions between different components.

- **Process Twin:** A Process Twin simulates and optimizes manufacturing or business processes. It helps in identifying inefficiencies and improving the workflow by visualizing the process steps and outcomes digitally.

- **Factory Twin:** A Factory Twin is a virtual model of a manufacturing facility. It includes the machinery, workflows, and operations within the factory, enabling better management, monitoring, and optimization of factory performance.


- **Supply Chain Twin:** A Supply Chain Twin is a comprehensive digital model of the entire supply chain network. It helps in monitoring and optimizing logistics, inventory, and overall supply chain efficiency by providing real-time insights and predictions.

- **Service Twin:** A Service Twin focuses on the service aspects of a product or system, including maintenance, repairs, and customer interactions. It ensures optimal service delivery by predicting service needs and improving customer experience.

- **Simulation/Validation using Digital Twins:** Simulation and validation using digital twins involve creating digital replicas of products, systems, or processes to test and validate their performance in a virtual environment. This helps in reducing the need for physical prototypes and accelerates the development process.


#### User Stories:

#### Digital Twin for Smart Scanning

- [Digital Twin for Smart Scanning](Modules/2.%20Learning%20Modules/2.8.%20Digital%20Twin%20for%20Smart%20Scanning/Digital%20Twin%20for%20Smart%20Scanning.ipynb) - A machine learning-based digital twin in metal additive manufacturing predicts future melt pools from existing data with high accuracy, enabling optimized smart scanning for quality production.

<center>
    <img src="img/02.jpg" alt="Alt text" width="400">
</center>

#### Assessment:
- [Take a quiz](http://example.com/quiz)

### Module 7: Digital Engineering Practice

#### User Stories:

#### <font color = 'red'>Module: Levi's Demo

#### Sensor Fusion Processing:
- [Sensor Fusion Processing](Modules/3.%20Advanced%20Topics/3.3.%20Sensor%20Fusion%20Processing/Sensor%20Fusion%20Processing.ipynb) - Selective laser melting (SLM) combines acoustic and photodiode signals for improved in-situ quality monitoring, significantly enhancing metal manufacturing reliability and precision through advanced sensor data analysis.

#### Predictive Maintenance Module
- [Predictive Maintenance Module](Modules/2.%20Learning%20Modules/2.2.%20Predictive%20Maintenance%20Module/Predictive%20Maintenance%20Module.ipynb) - Predictive maintenance is a prospective approach to maintenance that predicts equipment failures before they occur, minimizing downtime and maximizing productivity. Predictive maintenance models can identify patterns indicative of potential issues by analyzing historical data, such as sensor measurements and equipment performance. This allows maintenance teams to schedule repairs or replacements in advance, reducing costs associated with unplanned downtime and improving overall operational efficiency.

<center>
    <img src="img/PredictiveMaintenance.png" alt="Alt text" width="600">
</center>

##### <font color = 'red'>Future Plan: Prescriptive models

### Module 8: Digital Engineering Challenges 

Digital engineering has an ambitious scope: managing the entire lifecycle of a system of systems, including hardware systems as well as the software components. Many challenges exist for fully implementing digital engineering in government and industry. The most critical ones are interface and connectivity in the digital thread.

Course Evaluation: [Take Survey](http://example.com/quiz)

##  Modules

###  1. Fundamentals and Tools

####  1.1. Simple Linear Regression

- [Linear Regression](Modules/1.%20Fundamentals%20and%20Tools/1.1.%20Simple%20Linear%20Regression/Linear%20Regression.ipynb) - This module serves as a guide for forecasting time series data with simple linear regression.

####  1.2. Classification of categorical data

- [K-Nearest Neighbor](Modules/1.%20Fundamentals%20and%20Tools/1.2.%20Classification%20of%20categorical%20data/K-Nearest%20Neighbor.ipynb) - This module utilizes k-Nearest Neighbor (k-NN) Classification, a machine learning technique that classifies data points by looking at the most common class among their k closest neighbors, making predictions based on similarity to the training data.

- [Support Vector Machine](Modules/1.%20Fundamentals%20and%20Tools/1.2.%20Classification%20of%20categorical%20data/Support%20Vector%20Machine.ipynb) - This module utilizes Support Vector Machine (SVM), a machine learning technique that classifies data points by finding the optimal boundary that separates different classes, making predictions based on this boundary.

- [XGBoost](Modules/1.%20Fundamentals%20and%20Tools/1.2.%20Classification%20of%20categorical%20data/XGBoost.ipynb) - This module utilizes XGBoost, a machine learning technique that builds an ensemble of decision trees through gradient boosting to create a strong predictive model, optimizing performance and accuracy by combining multiple weak models.

####  1.3. Time-series Forecasting

- [ARIMA](Modules/1.%20Fundamentals%20and%20Tools/1.3.%20Time-series%20Forecasting/ARIMA.ipynb) - This module utilizes the ARIMA (AutoRegressive Integrated Moving Average) model, a statistical analysis technique that combines autoregression, differencing, and moving averages to analyze and forecast time series data by capturing various temporal structures and trends.

- [Exponential Smoothing](Modules/1.%20Fundamentals%20and%20Tools/1.3.%20Time-series%20Forecasting/Exponential%20Smoothing.ipynb) - This module utilizes the Exponential Smoothing method, a time series forecasting technique that applies weighted averages of past observations with exponentially decreasing weights to predict future data points, capturing trends and seasonal patterns effectively.

- [Gradient Boost Regressor](Modules/1.%20Fundamentals%20and%20Tools/1.3.%20Time-series%20Forecasting/Gradient%20Boost%20Regressor.ipynb) - This module utilizes the Gradient Boosting Regressor, a machine learning technique that builds an ensemble of prediction models, typically decision trees, to create a strong predictive model through iterative training and combining predictions to minimize errors and improve accuracy.

- [Linear Regression](Modules/1.%20Fundamentals%20and%20Tools/1.3.%20Time-series%20Forecasting/Linear%20Regression.ipynb) - This module utilizes Multiple Linear Regression, a statistical method that models the relationship between a dependent variable and multiple independent variables by fitting a linear equation to observed data, enabling the prediction of outcomes based on the linear relationships.

- [MLP Regressor](Modules/1.%20Fundamentals%20and%20Tools/1.3.%20Time-series%20Forecasting/MLP%20Regressor.ipynb) - This module utilizes the MLP (Multi-Layer Perceptron) Regressor, a machine learning technique that uses a feedforward artificial neural network to model complex relationships between input and output data, enabling accurate predictions through learning multiple layers of representation.

- [Random Forest Regression](Modules/1.%20Fundamentals%20and%20Tools/1.3.%20Time-series%20Forecasting/Random%20Forest%20Regression.ipynb) - This module utilizes Random Forest Regression, a machine learning technique that builds an ensemble of decision trees on various subsets of the data and averages their predictions to improve accuracy and control overfitting.

####  1.4. Image Classification

- [Image Classification](Modules/1.%20Fundamentals%20and%20Tools/1.4.%20Image%20Classification/Image%20Classification.ipynb) - This module utilizes Convolutional Neural Networks (CNNs) for image classification, a deep learning technique that automatically learns to recognize patterns and features in images through multiple layers of convolution and pooling, enabling the classification of images into different categories.

###  2. Learning Modules

####  2.1. 3D Printing Image Processing

- [Image Processing](Modules/2.%20Learning%20Modules/2.1.%203D%20Printing%20Image%20Processing/Image%20Processing.ipynb) - This module utilizes various image processing techniques to analyze, enhance, and manipulate images, enabling tasks such as resizing, cropping, filtering, and edge detection.

####  2.2. Predictive Maintenance Module

- [Predictive Maintenance Module](Modules/2.%20Learning%20Modules/2.2.%20Predictive%20Maintenance%20Module/Predictive%20Maintenance%20Module.ipynb) - Predictive maintenance is a prospective approach to maintenance that predicts equipment failures before they occur, minimizing downtime and maximizing productivity. Predictive maintenance models can identify patterns indicative of potential issues by analyzing historical data, such as sensor measurements and equipment performance. This allows maintenance teams to schedule repairs or replacements in advance, reducing costs associated with unplanned downtime and improving overall operational efficiency.

####  2.3. Cost Estimation Analysis

- [Cost Estimation using GCode](Modules/2.%20Learning%20Modules/2.3.%20Cost%20Estimation%20Analysis/Cost%20Estimation%20using%20GCode.ipynb) - This modules of cost analysis calculates the cost of a 3D printed part which involves considering labor, printer operation costs, material choice, and the part’s volume and shape, all critical for determining total production costs and ensuring economic viability.

####  2.4. Effect of Infill Patterns on Sintering

- [Effect of Infill Patterns on Sintering](Modules/2.%20Learning%20Modules/2.4.%20Effect%20of%20Infill%20Patterns%20on%20Sintering/Effect%20of%20Infill%20Patterns%20on%20Sintering.ipynb) - This study module aims to extract actionable data from different infill patterns and densities in metal 3D printing with binder paste. The focus is on gathering empirical insights that can serve as inputs for predictive models of shrinkage during the sintering process.

####  2.5. Height Map Feature Extraction

- [3D Scan Feature Extraction](Modules/2.%20Learning%20Modules/2.5.%20Height%20Map%20Feature%20Extraction/3D%20Scan%20Feature%20Extraction.ipynb) - The described training module is focused on several key skills necessary for working effectively with 3D scans

####  2.6. Model Predictive Control

- [Model Predictive Control](Modules/2.%20Learning%20Modules/2.6.%20Model%20Predictive%20Control/Model%20Predictive%20Control.ipynb) - This module demonstrates calculating a single time step's move trajectory using the same system as the Dahlin controller.

####  2.7. Digital Twin Lithium-Ion Battery

- [Digital Twin Lithium-Ion](Modules/2.%20Learning%20Modules/2.7.%20Digital%20Twin%20Lithium-Ion%20Battery/Digital%20Twin%20Lithium-Ion.ipynb) - Hybrid digital twin of a Li-ion battery using a physics based model

####  2.8. Digital Twin for Smart Scanning

- [Digital Twin for Smart Scanning](Modules/2.%20Learning%20Modules/2.8.%20Digital%20Twin%20for%20Smart%20Scanning/Digital%20Twin%20for%20Smart%20Scanning.ipynb) - A machine learning-based digital twin in metal additive manufacturing predicts future melt pools from existing data with high accuracy, enabling optimized smart scanning for quality production.

###  3. Advanced Topics

####  3.1. Aerotech Data Collection Platform

- [Aerotech Data Collection](Modules/3.%20Advanced%20Topics/3.1.%20Aerotech%20Data%20Collection%20Platform/Aerotech%20Data%20Collection.ipynb) - This module preprocesses sensor data from a custom-built Fused Filament Fabrication (FFF) 3D printer for modeling the extrusion process, incorporating feedback on motion, temperature, pressure, force, and cooling dynamics. 

####  3.2. Material Properties Finite Element Analysis

- [Material Properties FEA](Modules/3.%20Advanced%20Topics/3.2.%20Material%20Properties%20Finite%20Element%20Analysis/Material%20Properties%20FEA.ipynb) - This module demonstrates how our Graph Neural Network accelerates finite element analysis by efficiently predicting material behavior under mechanical loads.

####  3.3. Sensor Fusion Processing

- [Sensor Fusion Processing](Modules/3.%20Advanced%20Topics/3.3.%20Sensor%20Fusion%20Processing/Sensor%20Fusion%20Processing.ipynb) - Selective laser melting (SLM) combines acoustic and photodiode signals for improved in-situ quality monitoring, significantly enhancing metal manufacturing reliability and precision through advanced sensor data analysis.